# Sample Model For Project Portfolio Selection Multi-Yr Budgeting

# Part 1 : Loading the data from a CSV file

In [1]:
import pandas as pd

In [2]:
proj_list= pd.read_csv('Scenario4_InputData.csv', index_col=['Index'])

In [3]:
proj_list = proj_list.drop(['Year_1','Year_2','Year_3','Year4_Onwards'], axis=1)
proj_list

,Programme,CAPEX,Type,Location,Region,BenefitPerYr
Index,,,,,,
1,CivilStructural,5.0,AssetIntegrity,Facility H,Zone3,2.0
2,Electrical,15.0,AssetIntegrity,Facility M,Zone5,3.0
3,Piping,10.0,AssetIntegrity,Facility E,Zone5,5.0
4,Instrumentation,5.0,AssetIntegrity,Facility E,Zone5,2.0
5,CivilStructural,10.0,Growth,Facility M,Zone5,4.0
6,Expansion,15.0,Growth,Facility C,Zone3,8.0
7,Debottlenecking,10.0,Growth,Facility M,Zone5,2.0
8,Revamp,2.0,Growth,Facility G,Zone1,1.0
9,Revamp,3.0,Growth,Facility F,Zone1,1.0


In [4]:
proj_list.dtypes

Programme        object
CAPEX           float64
Type             object
Location         object
Region           object
BenefitPerYr    float64
dtype: object

In [5]:
discountrate=0.05
#Annual CAPEX Constraints Yr1,2,3,4+ 
capexconstraints=[20,30,25,0] #Note final year 4 onwards is zero as any that do not fit in 1-3 will spill to future capex

# Part 2 Solving Linear Programming Via PuLP

In [6]:
import pulp

In [7]:
# 2.1 Create A Model

phasing = pulp.LpProblem("Maximise", pulp.LpMaximize)

YrOne = pulp.LpVariable.dicts("YrOne", proj_list.index, cat='Binary')
YrTwo = pulp.LpVariable.dicts("YrTwo", proj_list.index, cat='Binary')
YrThree = pulp.LpVariable.dicts("YrThree", proj_list.index, cat='Binary')
YrFour = pulp.LpVariable.dicts("YrFour", proj_list.index, cat='Binary')


In [8]:
# 2.2 Set The Objective Function                                                                                                         
phasing += pulp.lpSum(
    [YrOne[idx]*proj_list.loc[idx]["BenefitPerYr"]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3) +
    YrTwo[idx]*proj_list.loc[idx]["BenefitPerYr"]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3) +
    YrThree[idx]*proj_list.loc[idx]["BenefitPerYr"]*(1/(1+discountrate)**3) for idx in proj_list.index])


# 2.3 Set The Constraints 
for idx in proj_list.index:
    phasing += YrOne[idx] + YrTwo[idx] + YrThree[idx] + YrFour[idx]==1
    
phasing += sum([YrOne[idx] * proj_list.loc[idx]["CAPEX"] for idx in proj_list.index]) <= capexconstraints[0]
phasing += sum([YrTwo[idx] * proj_list.loc[idx]["CAPEX"] for idx in proj_list.index]) <= capexconstraints[1]
phasing += sum([YrThree[idx] * proj_list.loc[idx]["CAPEX"] for idx in proj_list.index]) <= capexconstraints[2]


In [10]:
# 2.4 Run The Solver(s)

%time phasing.solve() #equivalent to phasing.solve(pulp.PULP_CBC_CMD()) as CBC is PulP's default solver

#phasing.solve(pulp.GLPK_CMD())


#phasing.solve()
pulp.LpStatus[phasing.status]

# 2.5 Print our objective function value and Output Solution
print (pulp.value(phasing.objective))

for idx in proj_list.index:
    print(idx, proj_list.loc[idx]["Programme"], YrOne[idx].value(), YrTwo[idx].value(), YrThree[idx].value(), YrFour[idx].value())


Wall time: 176 ms
54.57294028722599
1 CivilStructural 0.0 1.0 0.0 0.0
2 Electrical 0.0 0.0 1.0 0.0
3 Piping 0.0 1.0 0.0 0.0
4 Instrumentation 1.0 0.0 0.0 0.0
5 CivilStructural 0.0 1.0 0.0 0.0
6 Expansion 1.0 0.0 0.0 0.0
7 Debottlenecking 0.0 0.0 0.0 1.0
8 Revamp 0.0 1.0 0.0 0.0
9 Revamp 0.0 1.0 0.0 0.0
10 Expansion 0.0 0.0 1.0 0.0


In [11]:
# 2.6 Convert output into user friendly output for viewing or downloading 
pulpsolution=pd.DataFrame(columns = ["Yr1", "Yr2", "Yr3", "Yr4_Onwards","CAPEXYr1","CAPEXYr2","CAPEXYr3","CAPEXYr4Onwards"])
for idx in proj_list.index:
    to_append = [YrOne[idx].value(), YrTwo[idx].value(), YrThree[idx].value(), YrFour[idx].value(),YrOne[idx].value()*proj_list.loc[idx]["CAPEX"],YrTwo[idx].value()*proj_list.loc[idx]["CAPEX"],YrThree[idx].value()*proj_list.loc[idx]["CAPEX"],YrFour[idx].value()*proj_list.loc[idx]["CAPEX"]]
    perprojectyear = pd.Series(to_append, index = pulpsolution.columns)
    pulpsolution = pulpsolution.append(perprojectyear, ignore_index=True)

pulpsolution.index += 1 
pulpsolution

pulpoutput = pd.concat([proj_list, pulpsolution], axis=1)
pulpoutput['NPV_Benefit_3YrHorizon'] = (pulpoutput["Yr1"]*pulpoutput["BenefitPerYr"]*(1/(1+discountrate)**1 + 1/(1+discountrate)**2 + 1/(1+discountrate)**3) + pulpoutput["Yr2"]*pulpoutput["BenefitPerYr"]*(1/(1+discountrate)**2 + 1/(1+discountrate)**3) + pulpoutput["Yr3"]*pulpoutput["BenefitPerYr"]*(1/(1+discountrate)**3))
pulpoutput['NPV_Benefit_3YrHorizon']=pulpoutput['NPV_Benefit_3YrHorizon'].astype(float).round(2)
pulpoutput

,Programme,CAPEX,Type,Location,Region,BenefitPerYr,Yr1,Yr2,Yr3,Yr4_Onwards,CAPEXYr1,CAPEXYr2,CAPEXYr3,CAPEXYr4Onwards,NPV_Benefit_3YrHorizon
1,CivilStructural,5.0,AssetIntegrity,Facility H,Zone3,2.0,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,3.54
2,Electrical,15.0,AssetIntegrity,Facility M,Zone5,3.0,0.0,0.0,1.0,0.0,0.0,0.0,15.0,0.0,2.59
3,Piping,10.0,AssetIntegrity,Facility E,Zone5,5.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,8.85
4,Instrumentation,5.0,AssetIntegrity,Facility E,Zone5,2.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.45
5,CivilStructural,10.0,Growth,Facility M,Zone5,4.0,0.0,1.0,0.0,0.0,0.0,10.0,0.0,0.0,7.08
6,Expansion,15.0,Growth,Facility C,Zone3,8.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,21.79
7,Debottlenecking,10.0,Growth,Facility M,Zone5,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,0.00
8,Revamp,2.0,Growth,Facility G,Zone1,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.77
9,Revamp,3.0,Growth,Facility F,Zone1,1.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.77
10,Expansion,7.0,Growth,Facility J,Zone4,2.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,0.0,1.73


In [12]:
CAPEX_Totals=[pulpoutput['CAPEXYr1'].sum(),pulpoutput['CAPEXYr2'].sum(),pulpoutput['CAPEXYr3'].sum(),pulpoutput['CAPEXYr4Onwards'].sum()]

CAPEX_CheckSum= pd.DataFrame()
CAPEX_CheckSum['Year'] = [1,2,3,4]
CAPEX_CheckSum['CAPEX Phasing'] = CAPEX_Totals
CAPEX_CheckSum['Constraints'] = capexconstraints
print('Maximised Benefit Over 3 Yr Horizon in NPV Terms =', round(pulpoutput['NPV_Benefit_3YrHorizon'].sum(),2))
print()
print(CAPEX_CheckSum.to_string(index = False))

Maximised Benefit Over 3 Yr Horizon in NPV Terms = 54.57

 Year  CAPEX Phasing  Constraints
    1           20.0           20
    2           30.0           30
    3           22.0           25
    4           10.0            0
